In [2]:
import json
import requests
import os
from datetime import datetime
import zipfile
import tempfile
from collections import defaultdict

In [3]:
JSON_FILE = "./memories_history.json"

In [5]:
# Output folder
output_dir = "snapchat_downloads_renamed"
os.makedirs(output_dir, exist_ok=True)

In [6]:
# Load JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [7]:
memories = data['Saved Media']

In [8]:
def extension_from_headers(headers, media_type):
    content_type = headers.get("Content-Type", "").lower()

    if "video/mp4" in content_type:
        return ".mp4"
    if "image/jpeg" in content_type:
        return ".jpg"
    if "image/png" in content_type:
        return ".png"
    if "image/webp" in content_type:
        return ".webp"
    if "application/zip" in content_type:
        return ".zip"

    # fallback if Snapchat does something weird
    if media_type == "Video":
        return ".mp4"
    if media_type == "Image":
        return ".jpg"

    return ""
    
def safe_filename(s):
    return "".join(c for c in s if c.isalnum() or c in "._- ").rstrip()

def download_file(url, out_path, cookies=None):
    r = requests.get(url, stream=True, cookies=cookies)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return r.headers.get("content-type")

In [12]:
for item in memories:
    url = item["Media Download Url"]
    media_type = item["Media Type"]

    # clean date for filename
    date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
    base_name = date.strftime("%Y-%m-%d_%H%M%S")

    response = requests.get(url, allow_redirects=True)
    response.raise_for_status()

    ext = extension_from_headers(response.headers, media_type)
    filename = os.path.join(output_dir, base_name + ext)
    
    with open(filename, "wb") as f:
        f.write(response.content)

    print("Saved:", filename)


Saved: snapchat_downloads_renamed\2025-12-31_162224.zip
Saved: snapchat_downloads_renamed\2025-12-31_161549.zip
Saved: snapchat_downloads_renamed\2025-12-29_154007.jpg
Saved: snapchat_downloads_renamed\2025-12-28_201420.mp4
Saved: snapchat_downloads_renamed\2025-12-28_195554.jpg


KeyboardInterrupt: 